In [1]:
import pandas as pd

In [5]:
data = pd.read_csv('finish_flats.csv')

## Пока данные еще сыроватые, не успел до конца почистить, но осталось совсем не много.

## Добавится еще две фичи: vip статус объяления и тип дома 

## Y столбцом будет ценновая группа объявления: 
### 1: price <= 10 000
### 2:  10 000 < price <= 15 000
### 3: 15 000 < price <= 20 000
### 4: price > 20 000

## href и address не являются фичами

In [6]:
data

,href,price,agency,commissionPercent,metroDistance,metroStation,adress,metroLine,apartmentType,apartmentSquare,apartmentFloor,buildingFloor,absoluteTime,relativeTime,latitude,longitude,postalCode,area
0,/nizhniy_novgorod/kvartiry/studiya_24.7_m_610_...,10000,NaN,0 %,2.4 км,Парк Культуры,"Нижний Новгород, микрорайон Мончегорский, Монч...",Автозаводская линия,Студия,24.7,6,10,21 сентября 12:17,1 неделю назад,56.230577,43.826602,NaN,Автозаводский район
1,/nizhniy_novgorod/kvartiry/2-k_kvartira_44_m_4...,18000,NaN,0 %,550 м,Стрелка,"Нижний Новгород, ул Козицкого, д. 5/2−105",None,2-к квартира,44.0,4,5,21 сентября 13:29,1 неделю назад,56.291479,44.047453,603122.0,Советский район
2,/nizhniy_novgorod/kvartiry/1-k_kvartira_36_m_5...,13000,Агентство,0 %,3.6 км,Двигатель Революции,"Нижний Новгород, Пятигорская улица, 19",Автозаводская линия,1-к квартира,36.0,5,9,21 сентября 12:17,1 неделю назад,56.265607,43.976935,603009.0,Приокский район
3,/nizhniy_novgorod/kvartiry/1-k_kvartira_30_m_2...,12000,NaN,0 %,500 м,Пролетарская,"Нижний Новгород, проспект Ленина",Автозаводская линия,1-к квартира,30.0,2,5,21 сентября 11:55,1 неделю назад,56.263027,43.909049,NaN,Ленинский район
4,/nizhniy_novgorod/kvartiry/3-k_kvartira_90_m_4...,35000,АГЕНТСТВО НЕДВИЖИМОСТИ «ROYAL HOUSE»,70 %,5.3 км,Горьковская,"Нижний Новгород, улица Родионова, 191",Автозаводская линия,3-к квартира,90.0,4,9,21 сентября 13:48,1 неделю назад,56.303536,44.079037,603163.0,Нижегородский район
5,/nizhniy_novgorod/kvartiry/3-k_kvartira_90_m_1...,40000,АГЕНТСТВО НЕДВИЖИМОСТИ «ROYAL HOUSE»,70 %,1.1 км,Горьковская,"Нижний Новгород, улица Тимирязева, 3к2",Автозаводская линия,3-к квартира,90.0,14,18,21 сентября 13:49,1 неделю назад,56.304450,43.997129,603022.0,Советский район
6,/nizhniy_novgorod/kvartiry/1-k_kvartira_55_m_3...,25000,АГЕНТСТВО НЕДВИЖИМОСТИ «ROYAL HOUSE»,70 %,700 м,Горьковская,"Нижний Новгород, Малая Ямская улица, 18к3",Автозаводская линия,1-к квартира,55.0,3,6,21 сентября 13:50,1 неделю назад,56.313768,43.983394,603000.0,Нижегородский район
7,/nizhniy_novgorod/kvartiry/2-k_kvartira_60_m_3...,30000,АГЕНТСТВО НЕДВИЖИМОСТИ «ROYAL HOUSE»,70 %,500 м,Горьковская,"Нижний Новгород, улица Белинского, 15, подъезд 2",Автозаводская линия,2-к квартира,60.0,3,19,21 сентября 13:48,1 неделю назад,56.310882,44.001459,603105.0,Советский район
8,/nizhniy_novgorod/kvartiry/3-k_kvartira_100_m_...,40000,Агентство,без комиссии,5.7 км,Горьковская,"Нижний Новгород, Казанское шоссе. В. Печеры. Н...",Автозаводская линия,3-к квартира,100.0,9,10,21 сентября 14:48,1 неделю назад,56.323498,44.000336,NaN,Нижегородский район
9,/nizhniy_novgorod/kvartiry/2-k_kvartira_55_m_7...,30000,АГЕНТСТВО НЕДВИЖИМОСТИ «ROYAL HOUSE»,70 %,5.4 км,Горьковская,"Нижний Новгород, улица Родионова, 192к5",Автозаводская линия,2-к квартира,55.0,7,19,21 сентября 13:45,1 неделю назад,56.307472,44.081166,603093.0,Нижегородский район
